In [2]:
import pandas as pd
import numpy as np
import os

from MOA_L1000 import validation, config
import MOA_L1000.dataprep as dataprep
import MOA_L1000.dataload as dataload
%load_ext autoreload
%autoreload 2

X_train, X_holdout, w_train, y_train, y_holdout = dataload.load_data_v1()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:

def load_prediction(config_id=1):
    """Load params for the run, load ensemble solution and holdout metrics"""
    fcn_config = eval("config.get_params_{}".format(config_id))
    params = fcn_config()
    directory = params.get("output_directory")
    dict_solutions = {'df_ensemble':None, 'df_ensemble_mean':None, 'df_ensemble_median':None}
    
    filename = os.path.join(directory, "df_solution_ensemble.h5")
    if os.path.exists(filename):
        df_ensemble = pd.read_hdf(filename, 'root').astype('float64')
        dict_solutions['df_ensemble'] = df_ensemble
    filename = os.path.join(directory, "df_solution_ensemble_mean.h5")
    if os.path.exists(filename):
        df_ensemble = pd.read_hdf(filename, 'root').astype('float64')
        dict_solutions['df_ensemble_mean'] = df_ensemble
    filename = os.path.join(directory, "df_solution_ensemble_median.h5")
    if os.path.exists(filename):
        df_ensemble = pd.read_hdf(filename, 'root').astype('float64')
        dict_solutions['df_ensemble_median'] = df_ensemble
        
    filename = os.path.join(directory, "holdout_metrics.csv")
    df_metrics = pd.read_csv(filename, sep="\t", index_col=0)
    return dict_solutions, df_metrics, params


def compute_precision_and_recall(df_solution, df_ground=None):
    """Load solution matrix. Load params for our model. Load
    ensemble solution for our model. Load holdout metrics for our model. """
    assert df_ground is not None
    df_g = df_ground
    df_s = df_solution.loc[df_g.index, :]
    precisions = validation.compute_precision(df_s, df_g)
    recalls = validation.compute_recall(df_s, df_g)
    return precisions, recalls


def load_all_predictions_and_metrics(config_ids = []):
    """ For all solutions, load the ensemble solution, 
     and the holdout metrics"""
    solutions = {}
    metrics = {}
    for config_id in config_ids:
        dict_solutions, df_metrics, params = load_prediction(config_id)
        solutions[config_id] = dict_solutions
        metrics[config_id] = df_metrics
    
    df_metrics = pd.DataFrame(
        {config_id:d['holdout_score'] for config_id, d in metrics.items()}
        )
    return solutions, df_metrics

In [7]:
config_ids = ['DNN1', "DNN2", "DNN3", "DNN4", "DNN5", "DNN6", "DNN7", "DNN8", "DNN9", "DNN10", "DNN11", "DNN12", "DNN13", "DNN14"]
predictions, df_metrics = load_all_predictions_and_metrics(config_ids)
df_metrics

,DNN1,DNN2,DNN3,DNN4,DNN5,DNN6,DNN7,DNN8,DNN9,DNN10,DNN11,DNN12,DNN13,DNN14
0_0,0.05419,0.05138,0.04700,0.04875,0.05168,0.04791,0.04737,0.04630,0.05253,0.05565,0.05407,0.05704,0.05017,0.05030
0_1,0.05362,0.05431,0.04883,0.04779,0.05188,0.04760,0.04857,0.04699,0.05220,0.05718,0.05807,0.05932,0.05277,0.05157
0_2,0.05524,0.05703,0.04719,0.04905,0.05018,0.04853,0.04735,0.04687,0.05496,0.05593,0.05611,0.05926,0.05373,0.05347
0_3,0.05382,0.05237,0.04845,0.04695,0.05395,0.04773,0.04839,0.04590,0.05440,0.05552,0.05444,0.05828,0.05228,0.05075
0_4,0.05772,0.05282,0.04834,0.04952,0.05258,0.04770,0.04821,0.04710,0.05342,0.05659,0.05691,0.05918,0.05280,0.05210
1_0,0.05342,0.05590,0.04570,0.04850,0.05531,0.04816,0.04835,0.04750,0.05415,0.05612,0.05799,0.05654,0.05126,0.05152
1_1,0.05651,0.05424,0.04562,0.04916,0.05109,0.04759,0.04805,0.04488,0.05178,0.05755,0.05786,0.05939,0.05214,0.05316
1_2,0.05306,0.05203,0.04630,0.04851,0.05099,0.04810,0.04845,0.04696,0.05402,0.05604,0.05372,0.06003,0.05310,0.05182
1_3,0.05940,0.05131,0.04625,0.04772,0.05280,0.04751,0.04935,0.04656,0.05581,0.05571,0.05588,0.05792,0.05282,0.05031
1_4,0.04976,0.05479,0.04901,0.04934,0.05033,0.04847,0.04681,0.04581,0.05356,0.05936,0.05044,0.05782,0.05058,0.05110


In [8]:
dict_solutions, df_metrics, params = load_prediction('DNN2')
df_preds = dict_solutions['df_ensemble_median']
precisions, recalls = compute_precision_and_recall(df_preds, y_holdout)


In [9]:
precisions.tail(30).reset_index()

,moa,is_top
0,ACETYLCHOLINE RECEPTOR ANTAGONIST,0.100000
1,DNA INHIBITOR,0.100000
2,SODIUM CHANNEL INHIBITOR,0.111111
3,ANDROGEN RECEPTOR ANTAGONIST,0.117647
4,PDGFR INHIBITOR,0.125000
5,PHOSPHODIESTERASE INHIBITOR,0.150000
6,STEROL DEMETHYLASE INHIBITOR,0.200000
7,PROSTANOID RECEPTOR ANTAGONIST,0.250000
8,EGFR INHIBITOR,0.259259
9,PPAR RECEPTOR AGONIST,0.300000


In [10]:
recalls.tail(30).reset_index()

,moa,0
0,PROSTANOID RECEPTOR ANTAGONIST,0.166667
1,SEROTONIN RECEPTOR ANTAGONIST,0.185185
2,ANDROGEN RECEPTOR ANTAGONIST,0.222222
3,ADRENERGIC RECEPTOR ANTAGONIST,0.222222
4,PHOSPHODIESTERASE INHIBITOR,0.250000
5,DOPAMINE RECEPTOR ANTAGONIST,0.250000
6,ACETYLCHOLINE RECEPTOR ANTAGONIST,0.333333
7,STEROL DEMETHYLASE INHIBITOR,0.333333
8,P38 MAPK INHIBITOR,0.333333
9,VOLTAGE-GATED SODIUM CHANNEL BLOCKER,0.333333


In [11]:
df_preds

CCK RECEPTOR ANTAGONIST  PLK INHIBITOR  \
pert_id       pert_idose                                           
BRD-A02006392 1.11 uM                    0.000714       0.000108   
              10 uM                      0.001409       0.000270   
              3.33 uM                    0.000106       0.000245   
BRD-A03216249 1.11 uM                    0.047864       0.000534   
              10 uM                      0.020773       0.000321   
...                                           ...            ...   
BRD-K99792991 10 uM                      0.000362       0.000645   
              3.33 uM                    0.000328       0.000186   
BRD-K99964838 1.11 uM                    0.002361       0.000333   
              10 uM                      0.000246       0.001776   
              3.33 uM                    0.000173       0.000372   

                          TRICYCLIC ANTIDEPRESSANT  CHOLESTEROL INHIBITOR  \
pert_id       pert_idose                                                    
BRD-A02006392 1.11 uM                     0.001536               0.000296   
              10 uM                       0.001659               0.000672   
              3.33 uM                     0.002924               0.000777   
BRD-A03216249 1.11 uM                     0.001447               0.000926   
              10 uM                       0.008220               0.008295   
...                                            ...                    ...   
BRD-K99792991 10 uM                       0.000171               0.000075   
              3.33 uM                     0.001147               0.000191   
BRD-K99964838 1.11 uM                     0.005652               0.000267   
              10 uM                       0.004728               0.000042   
              3.33 uM                     0.001534               0.000085   

                          ATP CHANNEL BLOCKER  \
pert_id       pert_idose                        
BRD-A02006392 1.11 uM                0.000647   
              10 uM                  0.000808   
              3.33 uM                0.000566   
BRD-A03216249 1.11 uM                0.001212   
              10 uM                  0.002569   
...                                       ...   
BRD-K99792991 10 uM                  0.000303   
              3.33 uM                0.000148   
BRD-K99964838 1.11 uM                0.000280   
              10 uM                  0.000061   
              3.33 uM                0.000079   

                          THROMBOXANE RECEPTOR ANTAGONIST  SRC INHIBITOR  \
pert_id       pert_idose                                                   
BRD-A02006392 1.11 uM                            0.000938       0.001012   
              10 uM                              0.000604       0.001499   
              3.33 uM                            0.001004       0.000321   
BRD-A03216249 1.11 uM                            0.000769       0.000254   
              10 uM                              0.006444       0.000408   
...                                                   ...            ...   
BRD-K99792991 10 uM                              0.000130       0.000602   
              3.33 uM                            0.001076       0.003387   
BRD-K99964838 1.11 uM                            0.001877       0.011123   
              10 uM                              0.000184       0.045817   
              3.33 uM                            0.000430       0.007401   

                          AROMATASE INHIBITOR  OPIOID RECEPTOR ANTAGONIST  \
pert_id       pert_idose                                                    
BRD-A02006392 1.11 uM                0.000916                    0.000595   
              10 uM                  0.000934                    0.002014   
              3.33 uM                0.001635                    0.001821   
BRD-A03216249 1.11 uM                0.000538                    0.001535   
              10 uM                  0.000473     